<a href="https://colab.research.google.com/github/pycriador/pyDataGen/blob/main/pyDataGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Projeto de Setembro

Um programa simples para criar bases com valores aleatórios e com repetições controladas.

Carregando o objeto com suas funções e as principais bibliotecas do projeto

In [ ]:
!pip install faker

from faker import Faker
import unicodedata
import re

faker = Faker('pt-BR')

class GenFaker():

  def normalize(self, word):
    nfkd = unicodedata.normalize('NFKD', word)
    final_word = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    return re.sub('[^a-zA-Z0-9 \\\]', '', final_word)

  def faker_names(self, num):
    data = []
    for x in range(num):
      name = f"{faker.first_name()} {faker.last_name()}"
      data.append(name)
    return data

  def faker_email(self, num):
    data = []
    for x in range(num):
      data.append(faker.ascii_company_email())
    return data

  def create_email(self, nome=None, sufixo=None):
    nome = ''.join(nome.split('.'))
    nome = self.normalize(nome)
    lista = nome.split()
    if len(lista) > 1:
      email = f'{lista[0].lower()}.{lista[-1].lower()}@{sufixo}'
    else:
      email = f'{lista[0].lower()}@{sufixo}'

    return email

  def faker_estados(self, num):
    data = []
    for x in range(num):
      data.append(faker.administrative_unit())
    return data

  def faker_cores(self, num):
    data = []
    for x in range(num):
      data.append(faker.safe_color_name().capitalize())
    return data

app = GenFaker()

Criando a primeira leva de valores aleatórios, no exemplo criei 50 nomes, 50 emails, 50 estados, 50 cores de produtos, 5 produtos e dei 5 valores para esses produtos.

Lembrando que todos os valores aleatórios podem repetir, principalmente os estados e cores. Isso é uma coisa boa para gerar as bases.

In [4]:
from random import randrange, choice

#Criar lista de nomes
nomes = app.faker_names(50)
#Com base na lista de nomes, criar os e-mails
#emails = app.faker_email(len(nomes))

#Ou crirar os e-mails com base no 1° nome e o ultimo nome
emails = []
for nome in nomes:
  email = app.create_email(nome=nome, sufixo="email.com.br")
  emails.append(email)

#Com base na lista de nomes, criar estados
estados = app.faker_estados(len(nomes))
#Lista com cores dos produtos
cores = app.faker_cores(len(nomes))
#Número máximo de vendas
num_vendas = 100
#Lista de produtos
lista_produtos = ['Camisa', 'Sapato', 'Meia', 'Blusa', 'Boné']
#Valores dos produtos
valores_produtos = {'Camisa': 65, 'Sapato': 120, 'Meia': 5, 'Blusa': 100, 'Boné': 40}

#Criando lista com os produtos
produtos = []
for produto in range(len(nomes)):
  produtos.append(choice(lista_produtos))

#Criando lista com as quantidades de vendas (iniciais)
quantidades = []
for quantidade in range(len(nomes)):
  quantidades.append(randrange(num_vendas))

Crio um dataframe com todos os valores iniciais

In [5]:
import pandas as pd

#A função zip, junta as 3 listas (na ordem que está) em um objeto zip
# A função list, transforma o objeto zip em uma lista
dados = list(zip(nomes, emails, estados, produtos, cores, quantidades))

#Transformar os dados em um DataFrame
df = pd.DataFrame(dados, columns = ['Nome', 'Email', 'Estado', 'Produto', 'Cor', 'Quantidade'])
df.head(10)

,Nome,Email,Estado,Produto,Cor,Quantidade
0,Henrique Barbosa,henrique.barbosa@email.com.br,Rio de Janeiro,Meia,Ciano,67
1,Enzo Moura,enzo.moura@email.com.br,Pernambuco,Boné,Azul,77
2,Lívia Nascimento,livia.nascimento@email.com.br,Minas Gerais,Camisa,Magenta,84
3,Mariane Lima,mariane.lima@email.com.br,Rio Grande do Sul,Meia,Roxo,58
4,Otávio Gonçalves,otavio.goncalves@email.com.br,São Paulo,Camisa,Azul,55
5,Luiz Fernando da Rocha,luiz.rocha@email.com.br,Alagoas,Blusa,Cinza,45
6,João Miguel Carvalho,joao.carvalho@email.com.br,Sergipe,Boné,Verde,93
7,Camila Silveira,camila.silveira@email.com.br,Pará,Meia,Ciano,29
8,Caio Peixoto,caio.peixoto@email.com.br,Bahia,Blusa,Azul escuro,51
9,Ana Beatriz Almeida,ana.almeida@email.com.br,Santa Catarina,Blusa,Ciano,20


Essa parte é a repetição controlada, onde atribuimos mais valores aleatórios para a base inicial, dando mais valores aleatórios para estados, produtos, cores e quantidade.

In [6]:
#Repetir os nomes, porém informando novos Estados, Produtos, Cores e Quantidade
#Número máximo de repetições por nome
num_max_repet = 30

df_result = pd.DataFrame()

for index, row in df.iterrows():
  for x in range(randrange(num_max_repet)):
    data = {}

    #Dados fixos
    data['Nome'] = row['Nome']
    data['Email'] = row['Email']
    #Dados aleatórios
    data['Estado'] = choice(estados)
    data['Produto'] = choice(produtos)
    data['Cor'] = choice(cores)
    data['Quantidade'] = randrange(num_vendas)

    #Concatenar os resultados desse loop com os valores já existentes
    df_result = pd.concat([df_result, pd.DataFrame.from_records([data])])

#Concatenar tudo na base principal
df = pd.concat([df_result, df])

Mapeando os valores unitários dos produtos e depois multiplicando a quantidade pelo valor

In [7]:
#Mapeando o valor unitário com base no valor do produto
df['Valor unitário'] = df['Produto'].map(valores_produtos)
#Multiplicar valor unitário com a quantidade
df['Total'] = df['Quantidade'] * df['Valor unitário']
df.head(10)

,Nome,Email,Estado,Produto,Cor,Quantidade,Valor unitário,Total
0,Henrique Barbosa,henrique.barbosa@email.com.br,Piauí,Meia,Verde,79,5,395
0,Henrique Barbosa,henrique.barbosa@email.com.br,São Paulo,Blusa,Ciano,52,100,5200
0,Henrique Barbosa,henrique.barbosa@email.com.br,Mato Grosso do Sul,Blusa,Verde claro,4,100,400
0,Henrique Barbosa,henrique.barbosa@email.com.br,Mato Grosso do Sul,Camisa,Verde,3,65,195
0,Henrique Barbosa,henrique.barbosa@email.com.br,Espírito Santo,Sapato,Preto,22,120,2640
0,Henrique Barbosa,henrique.barbosa@email.com.br,Rondônia,Camisa,Cinza,15,65,975
0,Henrique Barbosa,henrique.barbosa@email.com.br,Santa Catarina,Sapato,Magenta,39,120,4680
0,Henrique Barbosa,henrique.barbosa@email.com.br,Pará,Meia,Laranja,61,5,305
0,Henrique Barbosa,henrique.barbosa@email.com.br,Rio de Janeiro,Meia,Rosa,65,5,325
0,Henrique Barbosa,henrique.barbosa@email.com.br,Minas Gerais,Meia,Ciano,48,5,240


Exportanto o resultado em XLSX e CSV

In [8]:
#Exportar em CSV
df.to_csv(f'Resultado.csv', index=False, encoding='ISO-8859-1', sep=';')
#Exportar em XLSX
with pd.ExcelWriter(f'Resultado.xlsx') as writer:
  df.to_excel(writer, sheet_name='Resultado', index=False)

**BONUS!**

Exportando os valores como "tabela" no Excel, formatando a planilha final e apresentando um resultado visualmente mais bonito.

In [ ]:
!pip install xlsxwriter

import xlsxwriter

def export(output='OutputFile', sheet='Base'):
  def colToExcel(col):  # col is 1 based
      excelCol = str()
      div = col
      while div:
          (div, mod) = divmod(div - 1, 26)  # will return (x, 0 .. 25)
          excelCol = chr(mod + 65) + excelCol

      return excelCol


  #Output sem formatação
  #with pd.ExcelWriter(f'PANDAS_{output}{timestamp}.xlsx') as writer:
  #    self.result.to_excel(writer, sheet_name=sheet, index=False)

  #Output com formatação e tabela do Excel
  workbook = xlsxwriter.Workbook(f'{output}.xlsx', options={'nan_inf_to_errors': True})
  # Criar aba
  worksheet = workbook.add_worksheet(sheet)
  # Set the columns widths
  worksheet.set_column('A:Z', 15)
  workbook.formats[0].set_font_size(8)
  colunas = df.columns.tolist()
  n_l = ['header' for x in colunas]
  lines = len(df.index) + 1

  columns = []
  for x in colunas:
      for y in n_l:
          columns.append({y: x})
          break

  col = colToExcel(len(colunas))
  worksheet.add_table('A1:%s%s' % (col, lines), {'data': df.values.tolist(),
                                                  'columns': columns,
                                                  'style': 'Table Style Light 11',
                                                  'name': sheet})
  workbook.close()

export(output='Resultado_em_tabela', sheet='Resultado')